In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os.path
import numpy as np

FOLDER = 'memory-usage'
mem = pd.read_csv(os.path.join(FOLDER, 'memory_usage_table.csv'))
mem

In [ ]:
# https://blog.espressif.com/esp32-programmers-memory-model-259444d89387
# Heap je v RAM: celková veľkosť heap = DRAM size (320 kB) - .data (14,6kB) - .bss (41,2kB)
with open(os.path.join(FOLDER, 'idf-size.txt'), 'r') as fr:
    for line in fr:
        print(line, end='')

In [ ]:
pool = mem['Total'][0]
buffer_sizes = pool - mem.iloc[:,3:].T

print('Heap size:', pool)
buffer_sizes

In [ ]:
# Každý proces zvlášť 32 .. 1024
fig, ax = plt.subplots(1, 1, figsize=(8, 6))

processes = buffer_sizes.copy().T
processes = (processes - processes.shift(1)).T / 1024
ax.stackplot(processes.index, *[list(processes[i]) for i in processes.iloc[:,1:]])
ax.axhline(y=(pool - buffer_sizes[0][1]) / 1024, color='r', linestyle='-')
ax.set_xlabel('Veľkosť vyrovnávacej pamäte [B]')
ax.set_ylabel('Celková obsadenosť haldy [kB]')
ax.grid()
ax.legend(['imu', 'fft & buffers', 'wifi', 'x', 'y', 'z', 'logger'], loc='upper left')
#ax.set_title('Profil spotreby pamäti')

plt.show()
processes

In [ ]:
summary = buffer_sizes[7] / 1024
fig, ax = plt.subplots(1, 1, figsize=(7, 4))
ax.plot(summary.index, summary, marker='o')
ax.grid()
ax.set_xlabel('Veľkosť vyrovnávacej pamäte [B]')
ax.set_ylabel('Celková obsadenosť haldy [kB]')
ax.axhline(y=pool / 1024, color='r', linestyle='-')
print('Obsadená pamäť [kB]\n', summary)

plt.show()
print('Obsadená pamäť [%]\n', summary)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(7, 4))
summary = buffer_sizes[7] / pool * 100
ax.plot(summary.index, summary, marker='o')
ax.grid()
ax.set_xlabel('Veľkosť vyrovnávacej pamäte [B]')
ax.set_ylabel('Celková obsadenosť haldy [%]')
ax.axhline(y=100, color='r', linestyle='-')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 6))

ax.stackplot(processes.index, *[list(processes[i]) for i in processes.iloc[:,4:7]])
ax.set_xlabel('Veľkosť vyrovnávacej pamäte [B]')
ax.set_ylabel('Obsadená pamäť [kB]')
ax.grid()
ax.legend(['x', 'y', 'z'], loc='upper left')
#ax[1].set_title('Profil spotreby pamäti úlohami')

In [ ]:
sizes = buffer_sizes.copy().T
processes = (sizes - sizes.shift(1)).T / 1024
processes.iloc[:,1:].T